In [ ]:
import weaviate
import ijson
from tqdm import tqdm
import json
from weaviate.classes.config import Configure,Property,DataType,VectorDistances,Tokenization
from pyserini.search.lucene import LuceneSearcher
from weaviate.classes.query import Rerank, MetadataQuery
import time
import requests
client = weaviate.connect_to_local()
#connecting to weaviate client.

In [ ]:
#creating a collection for actual data trec
client.collections.create("trec_final",properties=[Property(name="docid",data_type=DataType.TEXT,skip_vectorization=True),
                                                            Property(name="url",data_type=DataType.TEXT,skip_vectorization=True),
                                                            Property(name="title",data_type=DataType.TEXT,skip_vectorization=True),
                                                            Property(name="segment",data_type=DataType.TEXT,tokenization=Tokenization.LOWERCASE),
                                                            Property(name="headings",data_type=DataType.TEXT,skip_vectorization=True)],
                                                            #Property(name="end_char",data_type=DataType.INT,skip_vectorization=True)],
                                                 vectorizer_config=[
                                                                # Set another named vector
                                                                # Configure.NamedVectors.text2vec_transformers(  # Use the "text2vec-openai" vectorizer
                                                                # name="title", source_properties=["segment"]    
                                                                # ) ,
                                                                Configure.NamedVectors.text2vec_transformers(  # Use the "text2vec-openai" vectorizer
                                                                name="segment", source_properties=["segment"]    
                                                                )     # Set the source property(ies)


    ])


In [ ]:
#adding json data to weaviate 
collection = client.collections.get("trec_final")

with open('/mnt/ssd1/pyserini_r_final_segments/topics_final_r_segments_unique_array.json', 'r', encoding='utf-8') as file:
    objects = ijson.items(file, 'item')
    
    with collection.batch.dynamic() as batch:
        for obj in objects:
            properties = {
                "docid": obj["docid"],
                "url": obj["url"],
                "title": obj["title"],
                "headings": obj["headings"],
                "segment": obj["segment"]
            }
            batch.add_object(properties)

In [ ]:
#final output file for submission
import json
query_batch_size = 1
input_file = '/home/rag/Desktop/project-rag-ss24/src/finale/test topics retreival/topics_rag24_test.json'
output_file = '/home/rag/Desktop/project-rag-ss24/src/finale/output_files/r_final.tsv'
counter=0
with open(input_file, 'r') as f:
    queries_dict = json.load(f)
collection=client.collections.get('trec_final')
def process_query_batch(query_batch):
    counter=0
    results = []
    for query_obj in query_batch:
        query = query_obj['query']
        # Assuming 'collection.query.hybrid' is a function that takes 'query' and returns a list of responses
        hybrid_text_response = collection.query.hybrid(query, limit=20, return_metadata=MetadataQuery(score=True, explain_score=True))
        
        for response in hybrid_text_response.objects:
            counter=counter+1
            with open(output_file, 'a') as out_f:
                # Write the query id and response docid as a tab-separated line
                out_f.write(f"{query_obj['id']}\t{response.properties.get('docid')}\t{counter}\t{response.metadata.score}\n")
        

for i in range(0, len(queries_dict), query_batch_size):
    query_batch = queries_dict[i:i + query_batch_size]
    process_query_batch(query_batch)